# Hierarchical HMM: Chords & Song Sections

## Data Cleaning & Augmentation
Simplifying chords down to 42: base note (A-G) + accidental + major/minor(dim).

In [ ]:
import pandas as pd 

# Write song sections to csv
df = pd.read_csv("hf://datasets/ailsntua/Chordonomicon/chordonomicon_v2.csv",usecols=["chords", "main_genre"])
pop_structure = df[df["main_genre"] == "pop"][["chords"]].copy()
pop_structure["chords"] = pop_structure["chords"].str.split(" ")

print(len(pop_structure)) # 85k

# remove all songs with no section tags
pop_structure = pop_structure[
    pop_structure["chords"].apply(
        lambda tokens: any(token.startswith("<") for token in tokens)
    )
]

print(len(pop_structure)) # 55k

/var/folders/c8/zh68_k6j30l283wd6sjg39280000gn/T/ipykernel_62647/1002697580.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("hf://datasets/ailsntua/Chordonomicon/chordonomicon_v2.csv",usecols=["chords", "main_genre"])


85185
55641


In [ ]:
def expand_sections(tokens):
    """
    Given token list ["<intro_1>", "C", "<verse_1>", "F", "C", "G7", ...]
    Returns list of replicated section labels ["intro", "intro", "verse", "verse", "verse", "verse", ...]
    """
    section_labels = []
    current_section = None

    for tok in tokens:
        if tok.startswith("<") and tok.endswith(">"):
            raw = tok[1:-1] # drop <>
            current_section = raw.split("_")[0] # intro_1 --> intro
        else:
            section_labels.append(current_section) # replicate current section label if it's a chord
    return section_labels


pop_structure["sections"] = pop_structure["chords"].apply(expand_sections)
pop_structure = pop_structure[["sections"]]
print(pop_structure.head(15))

                                             sections
0   [intro, verse, verse, verse, verse, verse, ver...
4   [intro, verse, verse, verse, verse, chorus, ch...
6   [intro, intro, intro, intro, intro, intro, ver...
7   [intro, intro, intro, intro, intro, intro, int...
8   [chorus, chorus, chorus, chorus, chorus, choru...
10  [intro, intro, intro, intro, verse, verse, ver...
24  [intro, intro, intro, intro, intro, verse, ver...
25  [intro, intro, intro, intro, intro, verse, ver...
26  [intro, intro, intro, intro, verse, verse, ver...
27  [intro, intro, intro, intro, intro, intro, int...
35  [chorus, chorus, chorus, chorus, chorus, choru...
48  [intro, intro, intro, intro, intro, intro, int...
50  [intro, intro, intro, verse, verse, verse, ver...
54  [intro, intro, intro, intro, intro, intro, ver...
60  [intro, verse, verse, verse, verse, verse, ver...


In [20]:
pop_structure.to_csv("chordonomicon_v2_sections.csv", index=False)

In [ ]:
unique_tags = set(
    tag
    for tags in pop_structure["sections"]
    for tag in tags
)

print(unique_tags)
print(len(unique_tags))

{'bridge', 'solo', 'verse', 'intro', 'outro', 'instrumental', 'interlude', 'chorus'}
8


In [21]:
# Simplifying chords down to set of 42
notes = ["A", "B", "C", "D", "E", "F", "G"]
accs = ["b", "s", ""]
all_notes_list = [note + acc for note in notes for acc in accs]

def simplify_chord(chord: str) -> str:
    """
    Removes chord quality from a chord.
    """
    for note in all_notes_list:
        if not chord.startswith(note):
            continue

        suffix = chord.removeprefix(note)
        if suffix.startswith("min") or suffix.startswith("dim"):
            return note + "min"
        else:
            return note

    if chord == "sC":
        return "Cs"

    return ""

In [22]:
# Write simplified chords to csv
df = pd.read_csv("hf://datasets/ailsntua/Chordonomicon/chordonomicon_v2.csv",usecols=["chords", "main_genre"])
pop_chords = df[df["main_genre"] == "pop"][["chords"]]
pop_chords["chords"] = pop_chords["chords"].str.split(" ")

# remove all songs with no section tags, same as above
pop_chords = pop_chords[
    pop_chords["chords"].apply(
        lambda tokens: any(token.startswith("<") for token in tokens)
    )
]

print(len(pop_chords)) # 55k to match above

pop_chords["chords"] = pop_chords["chords"].map(
    lambda chords: [simplify_chord(chord) for chord in chords if not chord.startswith("<")]
)

print(pop_chords.head(15))

/var/folders/c8/zh68_k6j30l283wd6sjg39280000gn/T/ipykernel_62647/3221364142.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("hf://datasets/ailsntua/Chordonomicon/chordonomicon_v2.csv",usecols=["chords", "main_genre"])


55641
                                               chords
0   [C, F, C, E, Amin, C, F, C, G, C, F, C, E, Ami...
4   [C, G, C, G, C, F, Dmin, G, Dmin, G, C, G, C, ...
6   [G, Bmin, Amin, D, G, Bmin, Amin, D, G, Emin, ...
7   [Fsmin, Fs, B, E, Fs, B, E, Fsmin, B, As, Gsmi...
8   [C, Amin, Dmin, G, C, G, Amin, Dmin, G, C, Dmi...
10  [C, C, C, C, G, D, Emin, D, Emin, Amin, D, G, ...
24  [Amin, Amin, F, Es, E, Amin, Amin, F, E, Amin,...
25  [Emin, Emin, C, Bs, B, Emin, Emin, C, Bs, B, A...
26  [A, D, A, D, A, D, A, D, A, D, A, G, A, D, A, ...
27  [Amin, As, Amin, Amin, F, E, F, Amin, F, G, E,...
35  [Eb, Bb, Gmin, F, Eb, Bb, Gmin, F, Eb, Bb, Gmi...
48  [Amin, D, G, D, Emin, Amin, D, G, C, G, Amin, ...
50  [C, C, G, C, Emin, Amin, Gmin, F, C, Dmin, G, ...
54  [D, C, Bb, F, Dmin, C, Dmin, C, Bb, F, Dmin, C...
60  [D, Fsmin, G, A, D, Fsmin, G, A, D, Fsmin, G, ...


In [29]:
unique_chords = set(
    chord
    for chords in pop_chords["chords"]
    for chord in chords
)

print(unique_chords)
print(len(unique_chords))

notes = ["A", "B", "C", "D", "E", "F", "G"]
accs = ["b", "s", ""]
third = ["", "min"]
all_chords = [note + acc + t for note in notes for acc in accs for t in third]

missing = set(all_chords) - unique_chords
if missing:
    print(f"Not observed in dataset: " + str(missing))

{'Eb', 'Dsmin', 'Db', 'Fmin', 'Amin', 'Gsmin', 'Fsmin', 'Gs', 'Bbmin', 'As', 'Cs', 'Bb', 'F', 'Ds', 'B', 'Dmin', 'Ebmin', 'A', 'G', 'Fs', 'D', 'Gmin', 'Gb', 'Es', 'Emin', 'Asmin', 'Bmin', 'Abmin', 'E', 'Cmin', 'Dbmin', 'C', 'Ab', 'Csmin', 'Gbmin', 'Bs'}
36
Not observed in dataset: {'Cb', 'Esmin', 'Cbmin', 'Bsmin', 'Fbmin', 'Fb'}


In [30]:
pop_chords.to_csv("chordonomicon_v2_simplified.csv", index=False)

## Combined

In [7]:
import pandas as pd

notes = ["A", "B", "C", "D", "E", "F", "G"]
accs = ["b", "s", ""]
all_notes_list = [note + acc for note in notes for acc in accs]

def simplify_chord(chord: str) -> str:
    """
    Convert chord like Csmin, Abdim, G, Fsmin → Csmin, Abmin, G, Fsmin.
    Remove quality except treat min/dim as minor.
    """
    for note in all_notes_list:
        if chord.startswith(note):
            suffix = chord[len(note):]
            if suffix.startswith(("min", "dim")):
                return note + "min"
            else:
                return note

    if chord == "sC":
        return "Cs"

    return ""


def expand_sections(tokens):
    """
    Produce a section label for each actual chord token.
    """
    section_labels = []
    current_section = None

    for tok in tokens:
        if tok.startswith("<") and tok.endswith(">"):
            raw = tok[1:-1]
            current_section = raw.split("_")[0]
        else:
            section_labels.append(current_section)
    return section_labels


####### Main Processing #######

df = pd.read_csv(
    "hf://datasets/ailsntua/Chordonomicon/chordonomicon_v2.csv",
    usecols=["chords", "main_genre"]
)

# Filter only pop songs
pop_df = df[df["main_genre"] == "pop"].copy()
pop_df["tokens"] = pop_df["chords"].str.split(" ")

# Remove songs with no section tags
pop_df = pop_df[
    pop_df["tokens"].apply(lambda toks: any(tok.startswith("<") for tok in toks))
].copy()

print("After filtering:", len(pop_df))  # 55k

# Extract chords (simplified) and sections in aligned lists
pop_df["simple_chords"] = pop_df["tokens"].apply(
    lambda toks: [simplify_chord(tok) for tok in toks if not tok.startswith("<")]
)

pop_df["sections"] = pop_df["tokens"].apply(expand_sections)

# Sanity check: same length
assert all(
    len(c) == len(s)
    for c, s in zip(pop_df["simple_chords"], pop_df["sections"])
), "Mismatch in chord-section lengths!"

print(pop_df.head(10))

/var/folders/c8/zh68_k6j30l283wd6sjg39280000gn/T/ipykernel_69655/2273511548.py:44: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


After filtering: 55641
                                               chords main_genre  \
0   <intro_1> C <verse_1> F C E7 Amin C F C G7 C F...        pop   
4   <intro_1> C <verse_1> G C G C <chorus_1> F Dmi...        pop   
6   <intro_1> G Bmin Amin D G Bmin <verse_1> Amin ...        pop   
7   <intro_1> Fsmin Fsno3d Bno3d E/B Fsno3d Bno3d ...        pop   
8   <chorus_1> C Amin Dmin G C G Amin Dmin G C <ve...        pop   
10  <intro_1> Cmaj7 C Cmaj7 C <verse_1> G D Emin D...        pop   
24  <intro_1> Amin Amin7/G Fmaj7 Esus4 E <verse_1>...        pop   
25  <intro_1> Emin Emin7 Cmaj7 Bsus4 B <verse_1> E...        pop   
26  <intro_1> A D A D <verse_1> A D A D A D A G <c...        pop   
27  <intro_1> Amin Asus2 Amin Amin7/G F E7 F <vers...        pop   

                                               tokens  \
0   [<intro_1>, C, <verse_1>, F, C, E7, Amin, C, F...   
4   [<intro_1>, C, <verse_1>, G, C, G, C, <chorus_...   
6   [<intro_1>, G, Bmin, Amin, D, G, Bmin, <verse_...   


In [ ]:
# Extract chords by section type -> 8 dataframes
import pandas as pd

# get all section labels
all_sections = set(
    tag
    for tags in pop_df["sections"]
    for tag in tags
)
print("Observed sections:", str(all_sections))
print(len(all_sections))

# section -> df with column "chords"
section_dfs = {}
for section in all_sections:
    song_chords_per_sect = []

    for chords, secs in zip(pop_df["simple_chords"], pop_df["sections"]):
        # indices where the section appears
        idxs = [i for i, s in enumerate(secs) if s == section]

        # don't record in df if the song doesn't have the section
        if not idxs:
            continue

        # extract the chords at these indices
        seq = [chords[i] for i in idxs]

        song_chords_per_sect.append(seq)

    section_dfs[section] = pd.DataFrame({"chords": song_chords_per_sect})


for section in all_sections:
    print(f"\nSection: {section}")
    print(section_dfs[section].head())
    print("Num rows:", len(section_dfs[section]))

Observed sections: {'intro', 'verse', 'interlude', 'chorus', 'solo', 'outro', 'bridge', 'instrumental'}
8

Section: intro
                                  chords
0                                    [C]
1                                    [C]
2            [G, Bmin, Amin, D, G, Bmin]
3  [Fsmin, Fs, B, E, Fs, B, E, Fsmin, B]
4                           [C, C, C, C]
Num rows: 36886

Section: verse
                                              chords
0  [F, C, E, Amin, C, F, C, G, C, F, C, E, Amin, ...
1                           [G, C, G, C, G, C, G, C]
2  [Amin, D, G, Emin, Amin, D, G, Emin, Amin, D, ...
3  [Dmin, C, Dmin, C, Amin, Dmin, G, Dmin, C, Dmi...
4   [G, D, Emin, D, Emin, Amin, D, G, G, C, C, Amin]
Num rows: 52380

Section: interlude
                                              chords
0                  [F, G, Emin, Amin, Dmin, G, C, G]
1  [C, F, G, Cmin, G, Cmin, D, G, Ab, Cmin, G, Cmin]
2  [E, E, B, E, E, E, A, B, E, E, B, E, E, A, B, ...
3  [Amin, G, C, Amin, G, C, G, Ami

# Learning
Compute n-gram counts using CountVectorizer library

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

def count_n_grams(data, n: int = 1) -> pd.DataFrame:
    word_vectorizer = CountVectorizer(
        ngram_range=(1, n),
        analyzer="word",
        token_pattern=r"(?u)\b\w+\b",
        lowercase=False,
    )

    sparse_matrix = word_vectorizer.fit_transform(
        data.map(lambda chords: " ".join(chords))
    )

    frequencies = sum(sparse_matrix).toarray()[0]

    df_all = pd.DataFrame(
        frequencies,
        index=word_vectorizer.get_feature_names_out(),
        columns=["count"],
    )

    return df_all.groupby(by=lambda chords: len(chords.split(" ")))

In [40]:
import itertools

# Calculate transition matrix probabilities
# alpha is additive smoothing
def compute_unigram_prob(n_gram_counts, vocab, alpha=1.0):
    unigram = n_gram_counts.get_group(1).copy()
    unigram = unigram.reindex(vocab, fill_value=0)
    vocab_size = len(vocab)
    total_count = unigram["count"].sum()

    probs = (unigram["count"] + alpha) / (total_count + alpha * vocab_size)
    df = pd.DataFrame([probs.values], 
                      index=[""],
                      columns=vocab)
    return df

# def compute_bigram_prob(n_gram_counts, alpha=1.0):
#     bigram = n_gram_counts.get_group(2)
#     bigram["evidence"] = bigram.index.map(lambda s: s.split()[0]) # get (n-1)-length evidence
#     bigram["next"] = bigram.index.map(lambda s: s.split()[1]) # next chords

#     full_index = pd.MultiIndex.from_product([all_chords_list, all_chords_list], names=["evidence", "next"])
#     bigram = bigram.set_index(["evidence", "next"])
#     bigram = bigram.reindex(full_index, fill_value=0)

#     evidence_counts = bigram["count"].groupby(level="evidence").transform("sum")
#     num_next = bigram.index.get_level_values("next").nunique()
#     bigram["prob"] = (bigram["count"] + alpha) / (evidence_counts + alpha * num_next)

#     # 2d dataframe
#     return bigram["prob"].unstack(fill_value=0.0)

def compute_ngram_prob(n_gram_counts, vocab, n : int = 2, alpha=1.0):
    ngram = n_gram_counts.get_group(n).copy()

    ngram["evidence"] = ngram.index.map(lambda s: " ".join(s.split()[:-1]))
    ngram["next"] = ngram.index.map(lambda s: s.split()[-1])
    
    # generate all possible (n-1)-length chord sequeneces
    all_evidence_seq = [" ".join(evidence) for evidence in itertools.product(vocab, repeat=(n - 1))]
    full_index = pd.MultiIndex.from_product([all_evidence_seq, vocab], names=["evidence", "next"])

    # reindex to (vocab_size^(n-1), n)
    ngram = ngram.set_index(["evidence", "next"])
    ngram = ngram.reindex(full_index, fill_value=0)

    # compute probs
    evidence_counts = ngram["count"].groupby(level="evidence").transform("sum")
    vocab_size = len(vocab)
    ngram["prob"] = (ngram["count"] + alpha) / (evidence_counts + alpha * vocab_size)

    # 2d df transition matrix
    return ngram["prob"].unstack(fill_value=0.0)

Compute n-gram counts for section labels

In [37]:
# Sections

n = 2 # S_t+1 only depends on S_t

pop_sections = pop_df['sections']
n_gram_cnts = count_n_grams(pop_sections, n)
for key, _ in n_gram_cnts:
    print(n_gram_cnts.get_group(key).sort_values(by='count'), "\n\n")

                count
solo            28782
instrumental    78982
interlude       79954
outro          237375
bridge         300348
intro          306352
verse         1775400
chorus        1881980 


                     count
solo intro               3
solo instrumental        6
outro interlude         10
outro bridge            13
instrumental solo       16
...                    ...
outro outro         212670
intro intro         268478
bridge bridge       272191
verse verse        1665286
chorus chorus      1759010

[64 rows x 1 columns] 




In [38]:
all_sections = list(all_sections)

# Section unigram probs
unigram_probs_sec = compute_unigram_prob(n_gram_cnts, all_sections)
print(unigram_probs_sec.shape)
unigram_probs_sec

(1, 8)


,intro,verse,interlude,chorus,solo,outro,bridge,instrumental
,0.065332,0.378616,0.017051,0.401345,0.006138,0.050622,0.064051,0.016844


In [41]:
# Section bigram probs
bigram_probs_sec = compute_ngram_prob(n_gram_cnts, all_sections, n=2)
print(bigram_probs_sec.shape)
bigram_probs_sec

(8, 8)


next,bridge,chorus,instrumental,interlude,intro,outro,solo,verse
evidence,,,,,,,,
bridge,0.910917,0.061929,0.002734,0.002172,0.000174,0.008296,0.001064,0.012714
chorus,0.010745,0.946252,0.003727,0.003581,0.000295,0.009316,0.000962,0.025122
instrumental,0.016229,0.030211,0.880371,0.000894,0.000396,0.011977,0.000217,0.059706
interlude,0.011745,0.029250,0.001019,0.882058,0.000264,0.008388,0.001132,0.066145
intro,0.000475,0.011479,0.000661,0.000609,0.878746,0.000560,0.000223,0.107248
outro,0.000066,0.000295,0.000548,0.000052,0.001748,0.996378,0.000126,0.000787
solo,0.011450,0.034279,0.000246,0.001124,0.000140,0.008324,0.912756,0.031680
verse,0.003046,0.052321,0.000990,0.001215,0.000050,0.001597,0.000249,0.940532


Compute n-gram counts for chords

In [26]:
# Vocabulary of all possible chords (flats consolidated with sharps)
notes = ["A", "B", "C", "D", "E", "F", "G"]
accs = ["b", "s", ""]
third = ["", "min"]
all_chords = [note + acc + t for note in notes for acc in accs for t in third]

In [ ]:
n = 3
section_n_gram_counts = {} # index: section_n_gram_counts["intro"][1]
section_n_gram_probs = {}

for sec, df_sec in section_dfs.items():
    print(f"Processing section: {sec} with {len(df_sec)} sequences")

    # Extract the Series of chord lists
    chord_series = df_sec["chords"]

    # Compute counts for all n-grams up to n
    counts_df = count_n_grams(chord_series, n)

    # Store counts by n (1 → unigrams, 2 → bigrams, ...)
    section_n_gram_counts[sec] = counts_df

    # Compute probabilities for each n
    section_n_gram_probs[sec] = {}

    # Unigrams
    unigram_probs = compute_unigram_prob(counts_df, all_chords)
    section_n_gram_probs[sec][1] = unigram_probs

    # Bigrams
    if n >= 2:
        # You would define this similarly to your unigram version
        bigram_probs = compute_ngram_prob(counts_df, all_chords, n=2)
        section_n_gram_probs[sec][2] = bigram_probs

    # Trigrams
    if n >= 3:
        trigram_probs = compute_ngram_prob(counts_df, all_chords, n=3)
        section_n_gram_probs[sec][3] = trigram_probs

############################################################
# Example usage
############################################################

print("Sections processed:", list(section_n_gram_counts.keys()))

# Access unigram probabilities for the chorus
chorus_unigrams = section_n_gram_probs["chorus"][1]
print("Chorus unigram probability shape:", chorus_unigrams.shape)

# Access trigram counts for bridge
bridge_trigrams = section_n_gram_counts["bridge"].get_group(3)   # optional


Processing section: intro with 36886 sequences
Processing section: verse with 52380 sequences
Processing section: interlude with 6994 sequences
Processing section: chorus with 49098 sequences
Processing section: solo with 2466 sequences
Processing section: outro with 24195 sequences
Processing section: bridge with 24215 sequences
Processing section: instrumental with 7163 sequences
Sections processed: ['intro', 'verse', 'interlude', 'chorus', 'solo', 'outro', 'bridge', 'instrumental']
Chorus unigram probability shape: (1, 42)


# Inference
Deterministic and probabilistic methods.

In [ ]:
import numpy as np

def deterministic_inference(evidence):
    # evidence: string of n-1 space-separated chords
    
    n = len(evidence.split()) + 1
    ngram_probs = unigram_probs if n == 1 else (bigram_probs if n == 2 else trigram_probs)

    if evidence not in ngram_probs.index:
        raise KeyError(f"Evidence '{evidence}' not found in {n}-gram table")
    
    row_probs = ngram_probs.loc[evidence]
    return row_probs.idxmax() # returns next chord w highest prob, if there are several, the first one in col order

def probabilistic_inference(evidence):
    # evidence: string of n-1 space-separated chords
    
    n = len(evidence.split()) + 1
    ngram_probs = unigram_probs if n == 1 else (bigram_probs if n == 2 else trigram_probs)

    if evidence not in ngram_probs.index:
        raise KeyError(f"Evidence '{evidence}' not found in {n}-gram table")
    
    row_probs = ngram_probs.loc[evidence]
    cdf = np.cumsum(row_probs.values) # create cumulative distribution over next possible chord

    # sample over dist
    seed = np.random.random()
    idx = np.searchsorted(cdf, seed)
    
    return row_probs.index[idx] # return probabilistically chosen next chord

In [ ]:
### test inference for bigram ###
seq = []

for _ in range(16):
    if len(seq) == 0:
        evidence = ""
    elif len(seq) == 1:
        evidence = seq[-1]
    else:
        evidence = " ".join(seq[-2:])

    next_chord = probabilistic_inference(evidence) # can change to deterministic_inference()
    seq.append(next_chord)

print(seq)

['Csmin', 'Gs', 'A', 'E', 'Bmin', 'D', 'A', 'Fsmin', 'Csmin', 'E', 'A', 'D', 'Fsmin', 'D', 'A', 'Fsmin']


# Evaluation
Evaluate log-likelihood of an n-gram given a song

In [105]:
def song_log_likelihood_ngram(song, n, ngram_probs):
    # song: list of chords in song
    # n: order of the n-gram model
    # ngram_probs: dict[context_tuple] -> dict[target] = P(target | context)
    # ex: trigram ngram_prob = dict[(chord1, chord2)] = {chord0:P,...,chordV:P}, dict[chord3] = P(chord3 | chord1, chord2)
    # vocab_size: 42 or 36?

    ll = 0.0
    if len(song) < n:
        return 0.0

    for t in range(n-1, len(song)):
        if n == 1:
            context = ""
        else:
            context = " ".join(song[t-(n-1):t])

        target = song[t]

        try:
            p = ngram_probs.loc[context, target]
        except KeyError:
            print(f"KeyError given evidence {context}")
            p = 1e-12

        if p <= 0:
            p = 1e-12
        
        ll += np.log(p)

    return ll

In [106]:
# test log-likelihood
print(song_log_likelihood_ngram(seq, 2, bigram_probs))

-35.438954926606584


In [ ]:
def top_k_accuracy_ngram(song, n, ngram_probs, k=5):
    # song: list of chords in song
    # n: order of the n-gram model
    # ngram_probs: DataFrame with index=evidence, columns=next chords
    # k: number of top predictions to consider
    
    correct = 0
    total = 0
    
    if len(song) < n:
        return 0.0
    
    for t in range(n-1, len(song)):
        if n == 1:
            context = ""
        else:
            context = " ".join(song[t-(n-1):t])
        
        target = song[t]
        
        try:
            prob_row = ngram_probs.loc[context]
            
            top_k_chords = prob_row.nlargest(k).index.tolist()
            
            if target in top_k_chords:
                correct += 1
            total += 1
            
        except KeyError:
            total += 1
    
    return correct / total if total > 0 else 0.0

In [ ]:
# Test on top-k accuracy
print(f"Top-1: {top_k_accuracy_ngram(seq, 2, bigram_probs, k=1):.4f}")
print(f"Top-3: {top_k_accuracy_ngram(seq, 2, bigram_probs, k=3):.4f}")
print(f"Top-5: {top_k_accuracy_ngram(seq, 2, bigram_probs, k=5):.4f}")